# Assignment 1

There are three tab-delimited data files:

1. seed_stock_data.tsv

2. gene_information.tsv

3. cross_data.tsv


#1 contains information about seeds in your genebank

#2 contains information about genes

#3 contains information about the crosses you have made

Each file begins with a heading line, followed by lines of data

**Task 1**

"Simulate" planting 7 grams of seeds from each of the records in the seed stock genebank
then you should update the genebank information to show the new quantity of seeds
that remain after a planting. The new state of the genebank
should be printed to a new file, using exactly the same format as the
original file seed_stock_data.tsv

-- if the amount of seed is reduced to zero or less than zero, then
a friendly warning message should appear on the screen. The amount
of seed left in the gene bank is, of course, not LESS than zero 


In [565]:
File.open('seed_stock_data.tsv').each do |stock|
  puts stock
end
stock = []
require 'csv'
CSV.open("seed_stock_data.tsv","r", {col_sep:"\t"}).each do |stock1|
stock << stock1
end
stock

Seed_Stock	Mutant_Gene_ID	Last_Planted	Storage	Grams_Remaining

A334	AT1G69120	5/7/2014	cama2	28

A348	AT4G36920	3/11/2013	cama25	12

B3334	AT3G54340	1/12/2014	cama18	22

A51	AT1G30950	9/2/2013	cama25	5

B52	AT5G20240	8/11/2012	cama16	7



[["Seed_Stock", "Mutant_Gene_ID", "Last_Planted", "Storage", "Grams_Remaining"], ["A334", "AT1G69120", "5/7/2014", "cama2", "28"], ["A348", "AT4G36920", "3/11/2013", "cama25", "12"], ["B3334", "AT3G54340", "1/12/2014", "cama18", "22"], ["A51", "AT1G30950", "9/2/2013", "cama25", "5"], ["B52", "AT5G20240", "8/11/2012", "cama16", "7"]]

In [588]:
class Stock

  attr_accessor :seed_ID  
  attr_accessor :mutant_ID
  attr_accessor :last_planted
  attr_accessor :storage
  attr_accessor :grams
  
  def initialize (params = {})
    @seed_ID = params.fetch(:seed_ID, "00000")
    @mutant_ID = params.fetch(:mutant_ID, "000000000")
    @last_planted = params.fetch(:last_planted, "00/00/0000")
    @storage = params.fetch(:storage, 'unknown place')
    @grams = params.fetch(:grams, "0")
  
  end
    
    def planting_seeds(grams_of_seed)
      before_planting = grams
    if before_planting == 0
       puts "There is no stock of #{seed_ID}"
    else
      last_planted = Date.today
      remain = grams.to_i - grams_of_seed
            if remain <= 0
        remain = 0
        puts "We planted #{before_planting} grams of #{seed_ID}, but we have run out of stock!"
      else 
        puts "We planted #{grams_of_seed} grams of #{seed_ID}."
      end
    end
  File.open("updated_db.tsv", "a"){
    |f| f.write("\r\n#{seed_ID}\t#{mutant_ID}\t#{last_planted}\t#{storage}\t#{remain}\t")
    }
  end
end

:planting_seeds

In [589]:
File.write("updated_db.tsv", 
  ["Seed_Stock", "Mutant_Gene_ID", "Last_Planted", "Storage", "Grams_Remaining"].join("\t"))
puts ""

In [590]:
for i in (1..5)
  plants = Stock.new(
    :seed_ID => stock[i][0], 
    :mutant_ID => stock[i][1], 
    :last_planted => stock[i][2], 
    :storage => stock[i][3], 
    :grams => stock[i][4]
    )
  puts plants.planting_seeds(7)
end

We planted 7 grams of A334.
37
We planted 7 grams of A348.
37
We planted 7 grams of B3334.
39
We planted 5 grams of A51, but we have run out of stock!
36
We planted 7 grams of B52, but we have run out of stock!
36


1..5

**Task 2**

Process the information in cross_data.tsv and determine which genes are
genetically-linked. To achieve this, you will have to do a Chi-square test
on the F2 cross data. If you discover genes that are linked, this information
should be added as a property of each of the genes (they are both linked to each
other).

In [467]:
File.open('cross_data.tsv').each do |cross|
  puts cross
end
cross = []
require 'csv'
CSV.open("cross_data.tsv","r", {col_sep:"\t"}).each do |cross1|
cross << cross1
end
cross

Parent1	Parent2	F2_Wild	F2_P1	F2_P2	F2_P1P2

A334	A348	110	33	38	11

A348	B3334	80	26	29	8

B3334	A51	99	36	34	12

A51	B52	152	26	22	2

B52	A334	75	22	30	7



[["Parent1", "Parent2", "F2_Wild", "F2_P1", "F2_P2", "F2_P1P2"], ["A334", "A348", "110", "33", "38", "11"], ["A348", "B3334", "80", "26", "29", "8"], ["B3334", "A51", "99", "36", "34", "12"], ["A51", "B52", "152", "26", "22", "2"], ["B52", "A334", "75", "22", "30", "7"]]

In [591]:
class Cross

  attr_accessor :parent1  
  attr_accessor :parent2
  attr_accessor :f2_wild
  attr_accessor :f2_p1
  attr_accessor :f2_p2
  attr_accessor :f2_p1p2
  attr_accessor :linkage
  
  def initialize (params = {})
    @parent1 = params.fetch(:parent1, "00000")
    @parent2 = params.fetch(:parent2, "00000")
    @f2_wild = params.fetch(:f2_wild, "0")
    @f2_p1 = params.fetch(:f2_p1, "0")
    @f2_p2 = params.fetch(:f2_p2, "0")
    @f2_p1p2 = params.fetch(:f2_p1p2, "0")
    @linkage = params.fetch(:linkage, "unknown")
  end
  
  def chisq (a)
    total = f2_wild.to_f + f2_p1.to_f + f2_p2.to_f + f2_p1p2.to_f
    wild_exp = total*9/16
    p1_exp = total*3/16
    p2_exp = total*3/16
    p1p2_exp = total*1/16
    sum = ((f2_wild.to_f - wild_exp.to_f)**2/wild_exp.to_f) + ((f2_p1.to_f - p1_exp.to_f)**2/p1_exp.to_f) +((f2_p2.to_f - p2_exp.to_f)**2/p2_exp.to_f) + ((f2_p1p2.to_f - p1p2_exp.to_f)**2/p1p2_exp.to_f)
    if sum <= 7.815
    else 
      puts "The genes #{parent1} and #{parent2} are linked with a chisquare score of #{sum}"
      linking = Gene.new(:linkage => 'Yes')
    end
  end
end

:chisq

In [592]:
for i in (1..5)
  hibrid = Cross.new(
    :parent1 => cross[i][0], 
    :parent2 => cross[i][1], 
    :f2_wild => cross[i][2], 
    :f2_p1 => cross[i][3], 
    :f2_p2 => cross[i][4],
    :f2_p1p2 => cross[i][5]
    )
  hibrid.chisq(a)
end

The genes A51 and B52 are linked with a chisquare score of 32.27942794279428


1..5

**BONUS SCORES**

-> +1% if your Gene Object tests the format of the Gene Identifier and rejects incorrect formats without crashing

 Arabidopsis gene identifiers have the format /A[Tt]\d[Gg]\d\d\d\d\d/

If the identifier isn't correct, then your code should stop with a helpful error message

-> +1% if you create an Object that represents your entire Seed Stock "database"

The object should have a #load_from_file($seed_stock_data.tsv)

The object should access individual SeedStock objects based on their ID (e.g. StockDatabase.get_seed_stock('A334')

The object should have a #write_database('new_stock_file.tsv')

**To get these extra scores, your code needs to demonstrate that the functions really work!**